In [1]:
import numpy as np 
import pandas as pd
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
import re
import math
import random
from tqdm import tqdm

max_features = 20000
maxlen = 100

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
def get_model():
    embed_size = 128
    model = Sequential()
    model.add(Embedding(max_features, embed_size, input_length=maxlen))
    model.add(LSTM(50, input_shape=(embed_size, 1)))
    model.add(Dropout(0.1))
    model.add(Dense(50, activation="relu",kernel_initializer='he_normal'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [3]:
batch_size = 32
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
callbacks_list = [early]

classes = ["toxic", "severe_toxic", "obscene" ,"threat", "insult" ,"identity_hate"]
#Iterate for 5 samples
for i in tqdm(range(0,5)):
    #load test datasets
    df_test = pd.read_csv('clean_test_wo_capital.csv')
    df_test = df_test.fillna("")
    del df_test["Unnamed: 0"]
    
    #Iterate for 6 classes
    for cls in classes:
        #load each sampled training datasets
        s = cls + str(i) + '.csv'
        print(s)
        df_train = pd.read_csv(s)
        df_train = df_train.fillna("")
        #Combine test and train datasets
        df_all = pd.concat([df_train,df_test])

        #Build feature vector
        tokenizer = text.Tokenizer(num_words=max_features)
        tokenizer.fit_on_texts(list(df_all["comment_text"]))
        features = tokenizer.texts_to_sequences(df_all["comment_text"])
        features = sequence.pad_sequences(features, maxlen=maxlen)
        
        model = None
        model = get_model()
        #Training and predict
        if(cls == "threat"):
            epochs = 5
        elif(cls == "identity_hate" or cls == "severe_toxic" ):
            epochs = 4
        else:
            epochs = 3
        model.fit(features[0:df_train.shape[0]], df_train[cls], batch_size=batch_size, epochs=epochs, validation_split=0.3, callbacks=callbacks_list)
        prob = model.predict(features[df_train.shape[0]:])
        df_test[cls] = prob
    #Generate submission file for each sample
    #Average them later
    del df_test["comment_text"]
    df_test.set_index('id',inplace=True)
    df_test.to_csv("submission" + "_lstm_2_20000" + str(i) +  ".csv")

  0%|          | 0/5 [00:00<?, ?it/s]

toxic0.csv
Train on 21928 samples, validate on 9398 samples
Epoch 1/3
21928/21928 [==============================] - 186s 8ms/step - loss: 0.2943 - acc: 0.8717 - val_loss: 0.3361 - val_acc: 0.8483
Epoch 2/3
21928/21928 [==============================] - 186s 8ms/step - loss: 0.1519 - acc: 0.9427 - val_loss: 0.3510 - val_acc: 0.8569
Epoch 3/3
21928/21928 [==============================] - 180s 8ms/step - loss: 0.1035 - acc: 0.9624 - val_loss: 0.7293 - val_acc: 0.7320
severe_toxic0.csv
Train on 2228 samples, validate on 955 samples
Epoch 1/4
2228/2228 [==============================] - 19s 9ms/step - loss: 0.4577 - acc: 0.7859 - val_loss: 0.6070 - val_acc: 0.7204
Epoch 2/4
2228/2228 [==============================] - 18s 8ms/step - loss: 0.1026 - acc: 0.9713 - val_loss: 0.2465 - val_acc: 0.9016
Epoch 3/4
2228/2228 [==============================] - 18s 8ms/step - loss: 0.0613 - acc: 0.9825 - val_loss: 0.4087 - val_acc: 0.8576
Epoch 4/4
2228/2228 [==============================] - 18s 8ms

 20%|██        | 1/5 [47:17<3:09:09, 2837.49s/it]

toxic1.csv
Train on 21872 samples, validate on 9375 samples
Epoch 1/3
21872/21872 [==============================] - 173s 8ms/step - loss: 0.2878 - acc: 0.8762 - val_loss: 0.3940 - val_acc: 0.7926
Epoch 2/3
21872/21872 [==============================] - 171s 8ms/step - loss: 0.1495 - acc: 0.9423 - val_loss: 0.3969 - val_acc: 0.8150
Epoch 3/3
21872/21872 [==============================] - 174s 8ms/step - loss: 0.0996 - acc: 0.9642 - val_loss: 0.5851 - val_acc: 0.8027
severe_toxic1.csv
Train on 2235 samples, validate on 958 samples
Epoch 1/4
2235/2235 [==============================] - 19s 9ms/step - loss: 0.4388 - acc: 0.8031 - val_loss: 0.3148 - val_acc: 0.9238
Epoch 2/4
2235/2235 [==============================] - 18s 8ms/step - loss: 0.1205 - acc: 0.9655 - val_loss: 0.3204 - val_acc: 0.8925
Epoch 3/4
2235/2235 [==============================] - 17s 8ms/step - loss: 0.0493 - acc: 0.9897 - val_loss: 0.3464 - val_acc: 0.8852
Epoch 4/4
2235/2235 [==============================] - 18s 8ms

 40%|████      | 2/5 [1:32:53<2:19:20, 2786.97s/it]

...

 60%|██████    | 3/5 [2:05:30<1:23:40, 2510.11s/it]

toxic3.csv
Train on 21869 samples, validate on 9373 samples
Epoch 1/3
21869/21869 [==============================] - 124s 6ms/step - loss: 0.3152 - acc: 0.8589 - val_loss: 0.4775 - val_acc: 0.7804
Epoch 2/3
21869/21869 [==============================] - 123s 6ms/step - loss: 0.1548 - acc: 0.9398 - val_loss: 0.2896 - val_acc: 0.8784
Epoch 3/3
21869/21869 [==============================] - 127s 6ms/step - loss: 0.1054 - acc: 0.9608 - val_loss: 0.5086 - val_acc: 0.8084
severe_toxic3.csv
Train on 2235 samples, validate on 959 samples
Epoch 1/4
2235/2235 [==============================] - 15s 7ms/step - loss: 0.4789 - acc: 0.7763 - val_loss: 0.3009 - val_acc: 0.9187
Epoch 2/4
2235/2235 [==============================] - 11s 5ms/step - loss: 0.1202 - acc: 0.9642 - val_loss: 0.3180 - val_acc: 0.8905
Epoch 3/4
2235/2235 [==============================] - 13s 6ms/step - loss: 0.0499 - acc: 0.9870 - val_loss: 0.3450 - val_acc: 0.8801
Epoch 4/4
2235/2235 [==============================] - 12s 5ms

 80%|████████  | 4/5 [2:38:34<39:38, 2378.57s/it]  

toxic4.csv
Train on 21898 samples, validate on 9386 samples
Epoch 1/3
21898/21898 [==============================] - 118s 5ms/step - loss: 0.2912 - acc: 0.8748 - val_loss: 0.3668 - val_acc: 0.8437
Epoch 2/3
21898/21898 [==============================] - 118s 5ms/step - loss: 0.1481 - acc: 0.9457 - val_loss: 0.3886 - val_acc: 0.8397
Epoch 3/3
21898/21898 [==============================] - 115s 5ms/step - loss: 0.1029 - acc: 0.9636 - val_loss: 0.4733 - val_acc: 0.8225
severe_toxic4.csv
Train on 2231 samples, validate on 957 samples
Epoch 1/4
2231/2231 [==============================] - 15s 7ms/step - loss: 0.4328 - acc: 0.7911 - val_loss: 0.4991 - val_acc: 0.8349
Epoch 2/4
2231/2231 [==============================] - 12s 5ms/step - loss: 0.1098 - acc: 0.9668 - val_loss: 0.2565 - val_acc: 0.9080
Epoch 3/4
2231/2231 [==============================] - 11s 5ms/step - loss: 0.0527 - acc: 0.9857 - val_loss: 0.2832 - val_acc: 0.9007
Epoch 4/4
2231/2231 [==============================] - 12s 5ms

100%|██████████| 5/5 [3:10:50<00:00, 2290.05s/it]
